In [1]:
import numpy as np

# Qiskit Certification Prep - Module 1.1: Define Pauli Operators

## Section 1.1.0: Review from Module 0.1
### Pauli Gates
Pauli gates are the basic quantum logic gates. Their matrix representations and actions are:

- **X (NOT gate)**:
  $X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$
  Swaps $|0⟩$ and $|1⟩$

- **Y**:
  $Y = \begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix}$
  Applies a bit and phase flip

- **Z**:
  $Z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}$
  Flips the phase of $|1⟩$

  

### Properties of the Hadamard Gate and Pauli Gates:
For $\sigma_i \in \{H, X, Y, Z\}$
- **Involutory**: $\sigma_i^2 =I$
- **Traceless**: $\text{Tr} (\sigma_i) =0$
- **Hermitian**: $\sigma_i = \sigma_i^\dagger$
- **Unitary**: $\sigma_i^\dagger \sigma_i = I$

### Common Identities:
- $X = HZH$
- $Z = HXH$
- $H^2 = I$
- $X^2 = Y^2 = Z^2 = I$
- $XYZ = iI$  → So $-iXYZ = I$

### Commutation / Anticommutation:
- $ZX = -XZ$
- $YX = -XY$
- $ZY = -YZ$

These identities arise from the non-commutative nature of quantum gates and are fundamental to quantum algorithms and error correction.

## Section 1.1.1 Pauli class

- Focus: `qiskit.quantum_info.Pauli` (tensor products of single-qubit Paulis with
  a global phase).
- IBM Docs: https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.Pauli
- Methods we will cover:  
  - `to_matrix`: unitary matrix representation.  
  - `to_instruction`: circuit `Instruction` for `QuantumCircuit`.  
  - `compose`: order-aware product to combine Paulis.  
- Additional class: `qiskit.quantum_info.SparsePauliOp` for sparse linear
  combinations of Paulis (e.g., Hamiltonians/observables).  
- Goal: translate between strings ↔ matrices ↔ circuit instructions, compose
  Paulis, and use `SparsePauliOp` efficiently.  


## Section 1.1.2 `Pauli.to_matrix` — quick explainer + example

- What it does: `Pauli.to_matrix()` returns the dense unitary matrix
  (NumPy `ndarray`) of a `Pauli` on $n$ qubits (shape $2^n \times 2^n$).  
- When to use:  
  - To verify algebra (e.g., compare with a Kronecker product you built).  
  - To compute overlaps/expectations with states (e.g., for small $n$).  
- Notes: preserves the global phase encoded in the `Pauli`.  



In [2]:
### Minimal examples

from qiskit.quantum_info import Pauli, Statevector
import numpy as np

# 1) Single-qubit example: check the matrix of 'Y'
p_y = Pauli('Y')
M_y = p_y.to_matrix()
print("Y matrix:\n", M_y)

# Cross-check with the known matrix
Y_known = np.array([[0, -1j], [1j, 0]])
print("Matches known Y?", np.allclose(M_y, Y_known))

# 2) Two-qubit example: 'XZ' equals X ⊗ Z
p_xz = Pauli('XZ')
M_xz = p_xz.to_matrix()
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
print("\nXZ equals X⊗Z ?", np.allclose(M_xz, np.kron(X, Z)))

# 3) Use case: expectation ⟨+| Z |+⟩ = 0
plus = Statevector.from_label('+')       # |+> = (|0> + |1>)/√2
p_z = Pauli('Z')
M_z = p_z.to_matrix()
exp_plus_Z = (plus.data.conj().T @ M_z @ plus.data).real
print("\nExpectation <+|Z|+>:", exp_plus_Z)



Y matrix:
 [[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]
Matches known Y? True

XZ equals X⊗Z ? True

Expectation <+|Z|+>: -2.2371143170757382e-17


## Section 1.1.3 `Pauli.to_instruction` — quick explainer + use case

- What it does: converts a `Pauli` into a circuit-level `Instruction`.  
- Why it’s useful: lets you drop a multi-qubit Pauli directly into a
  `QuantumCircuit`, then rely on transpilation to map it to the target backend.  
- Returns: an `Instruction` with the correct qubit count and global phase.  
- Typical workflow: build `Pauli` → `to_instruction()` → `QuantumCircuit.append`
  → `transpile` for your backend.  


The following example combines a few concepts we've seen before:
- Qubit ordering convention in Qiskit
- Rz(pi) is the same as the Pauli Z matrix
- How to apply `to_instruction` and `evolve` methods in a quantum circuit

In [10]:
### Minimal example + backend mapping

from qiskit.quantum_info import Pauli, Statevector
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

# 1) Build a 2-qubit Pauli and convert to an Instruction
p = Pauli("XZ")                # acts as X on q0 and Z on q1
instr = p.to_instruction()
print(type(instr), instr.num_qubits, instr.name)

# 2) Use it in a circuit
qc = QuantumCircuit(2, name="pauli_demo")
qc.append(instr, [0, 1])
print(qc)

# 3) Functional check: equals explicit X on q1 and Z on q0
qc_explicit = QuantumCircuit(2)
qc_explicit.x(1)
qc_explicit.z(0)

sv_in = Statevector.from_label("00")
sv_pauli = sv_in.evolve(qc)
sv_explicit = sv_in.evolve(qc_explicit)
print("Same final state?", sv_pauli.equiv(sv_explicit))

# 4) Transpile onto a (fake) backend to see native decomposition
backend = FakeSherbrooke()
t_qc = transpile(qc, backend)
print(t_qc)


<class 'qiskit.circuit.library.generalized_gates.pauli.PauliGate'> 2 pauli
     ┌────────────┐
q_0: ┤0           ├
     │  Pauli(XZ) │
q_1: ┤1           ├
     └────────────┘
Same final state? True
global phase: π/2
          ┌───────┐
q_0 -> 60 ┤ Rz(π) ├
          └─┬───┬─┘
q_1 -> 61 ──┤ X ├──
            └───┘  


## Section 1.1.4 `Pauli.compose` — quick explainer + use cases

- What it does: algebraic product of two Paulis, returning a new `Pauli`.  
- Order matters: by default `self.compose(other)` means `self @ other`.  
- Use `front=True` to flip order: computes `other @ self`.  
- Phases are tracked: products can acquire factors in `{±1, ±i}`.  
- Supports subsets with `qargs` for multi-qubit targeting.  


In [11]:
### Minimal examples (single-qubit)

from qiskit.quantum_info import Pauli

X, Y, Z, I = Pauli("X"), Pauli("Y"), Pauli("Z"), Pauli("I")

# Default order: self @ other
XZ = X.compose(Z)                 # equals XZ = -i Y
ZX = Z.compose(X)                 # equals ZX = +i Y

print("X ∘ Z  ->", XZ)            # shows phase and label, e.g. '-iY'
print("Z ∘ X  ->", ZX)            # e.g. 'iY'

# Sanity check: X ∘ X = I (no phase)
print("X ∘ X  ->", X.compose(X))  # 'I'


X ∘ Z  -> iY
Z ∘ X  -> -iY
X ∘ X  -> I


## Section 1.1.5 `qiskit.quantum_info.SparsePauliOp` — section overview + example

- Reference docs:  
  https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.SparsePauliOp  

- What it is: a sparse *sum of Pauli strings* with complex coefficients.  
  Each term is a `Pauli` label (e.g., `'ZI'`) with a coefficient `c ∈ ℂ`.  

- Why it exists: compact, vectorized storage for operator algebra on many  
  qubits (e.g., Hamiltonians and observables).  

- Key differences vs `Pauli`:  
  - `Pauli`: represents **one** Pauli string with a global phase.  
  - `SparsePauliOp`: represents **∑ₖ cₖ Pₖ**, a *linear combination* of  
    Pauli strings. Not necessarily unitary.  
  - `Pauli.to_instruction()` exists (unitary); `SparsePauliOp` generally  
    does **not** become a circuit instruction.  
  - `SparsePauliOp` supports efficient algebra (`compose`, `simplify`,  
    scalar multiply, addition) over many terms.  

- Common constructors:  
  - `SparsePauliOp.from_list([(label, coeff), ...])`  
  - `SparsePauliOp(PauliList([...]), coeffs=...)`  

- Typical uses:  
  - Encode Hamiltonians (e.g., $H = Z ⊗ I + I ⊗ Z + 0.5 \, X ⊗ X$).  
  - Compute expectations on small states via `.to_matrix()`.  
  - Algebraic transforms (commutation checks, grouping, simplification).  



In [7]:
### Minimal, runnable example — Hamiltonian and expectation

import numpy as np
from qiskit.quantum_info import SparsePauliOp, Statevector

# H = ZI + IZ + 0.5 XX on 2 qubits
H = SparsePauliOp.from_list([
    ("ZI", 1.0),
    ("IZ", 1.0),
    ("XX", 0.5),
])

print("SparsePauliOp terms:")
print(H)

# Convert to a dense matrix (OK for small qubit counts)
H_mat = H.to_matrix()
print("\nDense matrix shape:", H_mat.shape)

# Expectation on |00>
psi = Statevector.from_label("00").data   # numpy array
exp_00 = np.vdot(psi, H_mat @ psi).real
print("⟨00|H|00⟩ =", exp_00)

# Expectation on |++>
plus2 = Statevector.from_label("++").data
exp_pp = np.vdot(plus2, H_mat @ plus2).real
print("⟨++|H|++⟩ =", exp_pp)

# Algebra: scale and simplify
H2 = (2.0 * H).simplify()
print("\nScaled & simplified (coeffs doubled):")
print(H2)



SparsePauliOp terms:
SparsePauliOp(['ZI', 'IZ', 'XX'],
              coeffs=[1. +0.j, 1. +0.j, 0.5+0.j])

Dense matrix shape: (4, 4)
⟨00|H|00⟩ = 2.0
⟨++|H|++⟩ = 0.49999999999999994

Scaled & simplified (coeffs doubled):
SparsePauliOp(['ZI', 'IZ', 'XX'],
              coeffs=[2.+0.j, 2.+0.j, 1.+0.j])


## Section 1.1.6 Basic Multiple Choice Questions

## Section 1.1.7 Advanced Multiple Choice Questions